# 1. Presets

1. Run setup notebook!
2. Add Slim modules to Python Path

In [5]:
!pip install contextlib2
import os
slim_path = ":models/research/slim"
old_path = (os.environ.get("PYTHONPATH") or '')

if not (slim_path in old_path):
    new_python_path = old_path + slim_path

%env PYTHONPATH=$new_python_path

env: PYTHONPATH=:models/research/slim


# 2. Download and build visual wakeword dataset

The existing dataset (download via the setup notebook) is designed to be used for training models for localization, so the images aren't labeled with the "contains a X", "doesn't contain a X" categories that we want to train for. Instead each image comes with a list of bounding boxes for all of the objects it contains. "X" is one of these object categories, so to get to the classification labels we want, we have to look for images with bounding boxes for people. To make sure that they aren't too tiny to be recognizable we also need to exclude very small bounding boxes. Slim contains a script to convert the bounding box into labels:

In [ ]:
LABEL='person'
%cd ~
! python sparky-rooms/tensorflow-model/models/research/slim/download_and_convert_data.py \
--dataset_name=visualwakewords \
--dataset_dir=dataset/visualwakewords \
--small_object_area_threshold=0.005 \
--foreground_class_of_interest=$LABEL

In [8]:
%cd ~
! python sparky-rooms/tensorflow-model/models/research/slim/datasets/build_visualwakewords_data.py \
--logtostderr \
--train_image_dir=dataset/raw-data/train2014 \
--val_image_dir=dataset/raw-data/val2014 \
--train_annotations_file=dataset/raw-data/annotations/instances_train2014.json \
--val_annotations_file=dataset/raw-data/annotations/instances_val2014.json \
--output_dir=dataset/visual-wake-words \

/home/jupyter
python: can't open file 'sparky-rooms/tensorflow-model/models/research/slim/datasets/build_visualwakewords_data.py': [Errno 2] No such file or directory


# 3. Train

In [ ]:
! python models/research/slim/train_image_classifier.py \
    --train_dir=vww_96_grayscale \
    --dataset_name=visualwakewords \
    --dataset_split_name=train \
    --dataset_dir=dataset/visual-wake-words \
    --model_name=mobilenet_v1_025 \
    --preprocessing_name=mobilenet_v1 \
    --train_image_size=96 \
    --input_grayscale=True \
    --save_summaries_secs=300 \
    --learning_rate=0.045 \
    --label_smoothing=0.1 \
    --learning_rate_decay_factor=0.98 \
    --num_epochs_per_decay=2.5 \
    --moving_average_decay=0.9999 \
    --batch_size=96 \
    --max_number_of_steps=1000000